## Практическая работа

В этой практической работе четыре обязательные задачи.

*Обязательные задачи* нужно сделать для того, чтобы проверить, что вы действительно усвоили материал модуля. Сдайте их на проверку.

Удачи!

Цели практической работы: 


1.   Потренироваться в обучении моделей деревьев решений.
2.   Потренироваться в обучении моделей случайного леса. 
3.   Научиться оценивать качество моделей с помошью Accuracy и confusion matrix.
4.   Научиться увеличивать качество моделей с помощью тюнинга параметров.




Что оценивается:

*   Все пункты и критерия приёмки задания выполнены.
*   Удаление колонок по результатам feature_importance и определения типов производится с помощью кода, а не перечислением их вручную.




## Обязательные задачи

### Описание датасета:
- `id`: идентификатор записи;
- `is_manufacturer_name`: признак производителя автомобиля;

- `region_*`: регион;
- `x0_*`: тип топлива;
- `manufacturer_*`: производитель;
- `short_model_*`: сокращённая модель автомобиля;
- `title_status_*`: статус;
- `transmission_*`: коробка передач;
- `state_*`: штат;
- `age_category_*`: возрастная категория автомобиля;

- `std_scaled_odometer`: количество пройденных миль (после стандартизации);
- `year_std`: год выпуска (после стандартизации);
- `lat_std`: широта (после стандартизации);
- `long_std`: долгота (после стандартизации);
- `odometer/price_std`: отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`: количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`: количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`: длина наименования автомобиля (после стандартизации);
- `model_word_count_std`: количество слов в наименовании автомобиля (после стандартизации);
- `month_std`: номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`: день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`: количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);

- `price`: стоимость;
- `price_category`: категория цены.

0. *Подготовка базовой модели*

Обучите простую модель классификации с помощью DecisionTreeClassifier на данных из датасета vehicles_dataset_prepared.csv. Для этого сделайте следующие шаги:

1. Обучите модель дерева решений с зафиксированным random_state на тренировочной выборке.
2. Сделайте предикт на тестовой выборке.
3. Замерьте точность на тестовой выборке и выведите матрицу ошибок. 
4. Удалите фичи с нулевыми весами по feature_importance из тренировочной и тестовой выборок.
5. Заново обучите модель и измерьте качество.

In [5]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix 

In [13]:
df = pd.read_csv('vehicles_dataset_prepared.csv')

df_prepared = df.copy()
df_prepared = df_prepared.drop(['price', 'odometer/price_std'], axis=1)
x = df_prepared.drop(['price_category'], axis=1)
y = df_prepared['price_category']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [16]:
# Ваш код здесь
# Первоначальная модель
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
predicted_train = clf.predict(x_train)
predicted_test = clf.predict(x_test)
print(f"Точность на тестовой выборке = {accuracy_score(y_test, predicted_test)}")
print(confusion_matrix(y_test, predicted_test))
# Удаляем нулевые фичи
f_imp_list = sorted(list(zip(x_train.columns, clf.feature_importances_ )), reverse=True, key=lambda x: x[1])

f_imp_list = [i[0] for i in f_imp_list if i[1] == 0]
df_deleted_zeros = df_prepared.copy()
df_deleted_zeros = df_deleted_zeros.drop(f_imp_list, axis=1)
x_new = df_deleted_zeros.drop(['price_category'], axis=1)
y_new = df_deleted_zeros.price_category
x_new_train, x_new_test, y_new_train, y_new_test = train_test_split(x_new, y_new, test_size=0.3, random_state=42)
clf.fit(x_new_train, y_new_train)
predicted_new_train = clf.predict(x_new_train)
predicted_new_test = clf.predict(x_new_test)
print(f"Точность на новой тестовой выборке = {accuracy_score(y_new_test, predicted_new_test)}")
print(confusion_matrix(y_new_test, predicted_new_test))


Точность на тестовой выборке = 0.6742896742896742
[[740  50 207]
 [ 46 682 225]
 [186 226 524]]
Точность на новой тестовой выборке = 0.6781011781011781
[[742  55 200]
 [ 45 684 224]
 [183 222 531]]


1. *Подготовка модели случайного леса*

Обучите простую модель классификации с помощью RandomForestClassifier. Для этого сделайте следующие шаги:
1. На новых урезанных семплах тренировочной и тестовой выборок обучите модель случайного леса с зафиксированным random_state=50. 

2. Сделайте предикт и посчитайте точность модели и матрицу ошибок. Сравните с предыдущей моделью дерева решений. Есть ли случаи, когда модель из пункта 1 отрабатывает лучше, чем модель случайного леса?

In [114]:
# Ваш код здесь 
rf_clf = RandomForestClassifier()
x_rf_train, x_rf_test, y_rf_train, y_rf_test = train_test_split(x_new, y_new, test_size=0.3, random_state=50)
rf_clf.fit(x_rf_train, y_rf_train)

predicted_rf_test = rf_clf.predict(x_rf_test)
print(f"Точноть случайного леса на тестовых данных = {accuracy_score(y_rf_test, predicted_rf_test)}")

Точноть случайного леса на тестовых данных = 0.7546777546777547


2. *Тюнинг модели случайного леса*

Увеличьте точность модели на тестовом датасете RandomForestClassifier c помощью тюнинга параметров. 

Параметры, которые можно настраивать для увеличения точности:

```
    `bootstrap'
    'max_depth'
    'max_features'
    'min_samples_leaf'
    'min_samples_split'
    'random_state'
    'n_estimators'

```



С описанием каждого из параметров можно ознакомиться в документации:


https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Задание засчитывается, если значение метрики строго выше 0,76 на тестовом датасете.

In [128]:
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
rf_clf = RandomForestClassifier()

# Инициализиурем сетку для переборов параметров
param_grid = {
    'n_estimators': list(range(100, 501, 100)),
   'max_features': ['sqrt', 'log2'],
   'min_samples_leaf': list(range(1, 10))
}

randomized_search_rf  = RandomizedSearchCV(
   estimator=rf_clf,
   n_iter= 20,
   param_distributions=param_grid,
   scoring='accuracy',
   verbose=1,
   n_jobs=-1
)
randomized_search_rf .fit(x_rf_train, y_rf_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   scoring='accuracy', verbose=1)

In [131]:
# Ваш код здесь 
best_params = randomized_search_rf.best_params_ # {'n_estimators': 500, 'min_samples_leaf': 1, 'max_features': 'log2'}
print(best_params)
rf_clf = RandomForestClassifier(n_estimators=500, min_samples_leaf=1, max_features='log2')
rf_clf.fit(x_rf_train, y_rf_train)
pred = rf_clf.predict(x_rf_test )
accuracy_score(y_rf_test, pred)

{'n_estimators': 500, 'min_samples_leaf': 1, 'max_features': 'log2'}


0.7668052668052668

3. *Анализ влияния фичей на модель*

Во всех задачах до вы работали над подготовленным датасетом, где фичи были заранее извлечены из текстовых переменных, отскейлены и пропущены через One Hot Encoder. Сравним, какой была бы предсказательная способность модели, если бы мы использовали только сырые данные из исходного датасета. Для этого проделайте следующие шаги:

1. Загрузите датасет `vehicles_dataset_old.csv`.
2. Удалите из него переменную `price` и все строковые колонки. Дерево решений и случайный лес не умеют самостоятельно работать со строковыми значениями.
3. Сформируйте x_train и x_test так же, как они были сформированы в предыдущих заданиях.
4. Обучите свою лучшую модель случайного леса на новых данных и замерьте качество. Убедитесь, что оно ухудшилось.
5. Найдите три фичи, которые лучшим образом повлияли на предсказательную способность модели.

In [42]:
df_old = pd.read_csv('vehicles_dataset_old.csv')
#display(df_old.head())

# Ваш код здесь 
df_new = df_old.drop(['price'], axis=1)
y_old = df_new.price_category

df_new = df_new.drop(df_new.dtypes[df_new.dtypes == object].index,axis=1)
x_old = df_new
x_old_train, x_old_test, y_old_train, y_old_test = train_test_split(x_old, y_old, test_size=0.3, random_state=42)
rf_clf_old = RandomForestClassifier()
rf_clf_old.fit(x_old_train,  y_old_train)
pred = rf_clf_old.predict(x_old_test)
print(accuracy_score(y_old_test, pred))
print("Точность стала хуже :(")
important = sorted([(name, score) for name, score in zip(x_old_train.columns, rf_clf_old.feature_importances_)], key=lambda x: x[1], reverse=True)
print(important[:3])

0.6334026334026334
Точность стала хуже :(
[('odometer', 0.2677762894441347), ('year', 0.23357334685321735), ('id', 0.16836437585674632)]
